In [10]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [17]:
def getPts(pose_keypoint):
    xs = []
    ys = []
    for i in range(0,len(pose_keypoint)-2,3):
        xs.append(pose_keypoint[i])
        ys.append(pose_keypoint[i+1])
    
    return np.array(list(zip(xs,ys)))


def centroid(pts):
    return np.array([sum(pts[:,0])/(len(pts)),sum(pts[:,1])/(len(pts))])
            
def stdDev(centroid,pts):
    return pts - centroid

def d2(pt1,pt2):
    return np.sqrt((pt1[0] - pt2[0])**2 + (pt1[1] - pt2[1])**2)

In [18]:
def getRythmObject(pose_keypoint):
    pts = getPts(pose_keypoint)
    
    #get centroid withought [0,0]
    filtered_zeroes = np.array([x for x in pts if x[0] != 0 or x[1] != 0])
    mask = centroid(filtered_zeroes)
    
    for i in range(len(pts)):
        if np.array_equal(pts[i],np.array([0,0])):
            pts[i] = mask
            
    return (pts - mask)/np.mean([d2(x,mask) for x in pts])


def getSceneRythmObjects(scene):
    people = []
    for ppl in scene:
        people.append([x for x in getRythmObject(ppl)])
    return np.array(people)

In [19]:
def sceneHasZeroes(scene):
    for person in scene:
        for i in range(2,len(person),3):
            if person[i] == 0:
                return True
    return False

def personHasZeroes(person_keypoint):
    for i in range(2,len(person_keypoint),3):
        if person_keypoint[i] == 0:
            return True
    
    return False

In [20]:
def loadScene(frame,root,f_to_f_name):
    file_name = f_to_f_name[frame]
    df = pd.read_json(os.path.join(root, file_name))
    people = df['people'].values
    scene = [person['pose_keypoints_2d'] for person in people]
    return scene

In [21]:

def loadPoses(path='testposes18/',cleaning_method='skip_missing'):
    scenes = []
    frame_to_f_name = {}
    
    for root, dirs, files in os.walk(path, topdown=False):
        for f_name in files:
            frame_number = int(f_name.split('_')[1])
            frame_to_f_name[frame_number] = f_name
            
    video_id = frame_to_f_name[0].split('_')[0]
            
    min_frame = min(frame_to_f_name.keys())
    max_frame = max(frame_to_f_name.keys())
    min_slice_len = 30
    #max_frame=700

    slice_n = 0
    
    df = pd.DataFrame({"StartFrame":[],"EndFrame":[],"Poses":[]})
    
    frame = min_frame
    while frame <= max_frame:
        
        scene = loadScene(frame,root,frame_to_f_name)
        slice_number = 0
        if not sceneHasZeroes(scene):
            slice_s = []
            frame_s = []
            while (not sceneHasZeroes(scene)) and (frame <= max_frame):
                pts = getSceneRythmObjects(scene)
                #print(np.shape(pts))
                #print(pts)
                slice_s.append(pts)
                frame_s.append(frame)
                frame += 1
                if (frame <= max_frame):
                     scene = loadScene(frame,root,frame_to_f_name)
                else:
                    break        
            
            slice_s = np.array(slice_s)
            if len(frame_s) > min_slice_len:
                person_n = 0
                for i in range(np.shape(slice_s)[1]):
                    df = df.append({"VideoId":video_id,
                                    "SliceNumber":slice_n,
                                    "PersonIndex":person_n,
                                    "StartFrame":min(frame_s),
                                    "EndFrame":max(frame_s),
                                    "Poses":slice_s[:,i,:,:]},ignore_index=True)
                    person_n +=1
                slice_n +=1
                
        
        else:
            frame += 1
        
        
    #pd.DataFrame({'Frame'})
        
    
    
                
    return df

In [ ]:
df = loadPoses()

In [ ]:
df[['VideoId','StartFrame',
    'EndFrame','SliceNumber',
    'PersonIndex','Poses']].to_json('518111ee-5822-4c75-995a-66a5d94db77f.json')

In [ ]:
df['SliceNumber']